<a href="https://colab.research.google.com/github/sHarsha-24/LLM--NETWORKING/blob/main/LLM(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
sentence = input("Enter a sentence: ")


words = sentence.split()
print("Word Tokens:", words)


characters = list(sentence)
print("Character Tokens:", characters)

In [ ]:
!pip install transformers torch scikit-learn
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

# Load pre-trained Sentence Transformer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

# Example sentences
text1 = "I love machine learning"
text2 = "I enjoy studying artificial intelligence"

# Generate embeddings
emb1 = get_embedding(text1)
emb2 = get_embedding(text2)

# Cosine similarity
similarity = cosine_similarity(emb1, emb2)

print("Text Similarity Score:", similarity[0][0])


In [ ]:
 !pip install transformers torch scikit-learn
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd

# Sample Dataset (replace with CSV if needed)
data = {
    "text": [
        "Government launches new education policy",
        "Aliens landed in India yesterday",
        "Stock market shows steady growth",
        "Drinking cow urine cures cancer",
        "Scientists discover new planet"
    ],
    "label": [1, 0, 1, 0, 1]  # 1 = Real, 0 = Fake
}

df = pd.DataFrame(data)

# Load tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

# Dataset class
class FakeNewsDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding=True,
            max_length=128
        )
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    df["text"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    random_state=42
)

train_dataset = FakeNewsDataset(X_train, y_train)
test_dataset = FakeNewsDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Training loop (1 epoch – demo)
model.train()
for batch in train_loader:
    optimizer.zero_grad()
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

print("Training completed")

# Prediction
model.eval()
news = ["NASA confirms water on Mars"]
inputs = tokenizer(news, return_tensors="pt", truncation=True, padding=True)

with torch.no_grad():
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1)

print("Prediction:", "Real News" if prediction.item() == 1 else "Fake News")



In [ ]:
!pip install transformers torch
from transformers import pipeline

# Load sentiment analysis pipeline
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

# Sample texts
texts = [
    "I love this product, it works perfectly!",
    "This is the worst experience ever",
    "The movie was okay, not bad",
    "I am extremely happy with the service",
    "I hate waiting in long queues"
]

# Analyze sentiment
for text in texts:
    result = sentiment_analyzer(text)[0]
    print("Text:", text)
    print("Sentiment:", result["label"])
    print("Confidence:", round(result["score"], 3))
    print()
